### Notebook:
- Fits linear model (gam) to discovery and validation cohorts

In [23]:
library(mgcv)
library(oddsratio)

Loading required package: nlme
This is mgcv 1.8-23. For overview type 'help("mgcv-package")'.


# 1. Discovery cohort
- TCGA

##### 1A. Load discovery cohort

In [2]:
discovery_data = read.csv('../generated_data/gam_input.expressed_mutations.2.tsv', sep='\t')
cat(dim(discovery_data))
head(discovery_data[c('y..has_mutation.', 'centered_log_phbrI', 'centered_log_phbrII', 'centered_sex', 'patient_ids', 'disease')])

2199477 16

y..has_mutation.,centered_log_phbrI,centered_log_phbrII,centered_sex,patient_ids,disease
0,0.05987674,-1.152503930,-0.4230716,TCGA-02-0003,TCGA-GBM
0,-0.02521391,-1.676057048,-0.4230716,TCGA-02-0003,TCGA-GBM
0,0.40872167,0.001467257,-0.4230716,TCGA-02-0003,TCGA-GBM
0,0.79788495,0.822290563,-0.4230716,TCGA-02-0003,TCGA-GBM
0,0.44322198,0.169821203,-0.4230716,TCGA-02-0003,TCGA-GBM
0,-0.42148021,0.195804718,-0.4230716,TCGA-02-0003,TCGA-GBM


### Table 1

##### Age

In [5]:
g_age <- mgcv::gam(y..has_mutation. ~ centered_log_phbrI * centered_log_phbrII * centered_age, 
                   random=list(patient_ids=~1), 
                   family=binomial(link='logit'), 
                   data=discovery_data)
summary(g_age)$p.table

,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),-6.597925322,0.0201500717,-327.439298,0.000000e+00
centered_log_phbrI,0.043530899,0.0163642249,2.660126,7.811145e-03
centered_log_phbrII,0.311381693,0.0199872009,15.579055,1.010381e-54
centered_age,-0.002475534,0.0013064735,-1.894821,5.811607e-02
centered_log_phbrI:centered_log_phbrII,-0.025895182,0.0139694478,-1.853701,6.378192e-02
centered_log_phbrI:centered_age,-0.002963916,0.0010702749,-2.769304,5.617622e-03
centered_log_phbrII:centered_age,-0.003520486,0.0013069176,-2.693732,7.065696e-03
centered_log_phbrI:centered_log_phbrII:centered_age,0.002114323,0.0009114125,2.319831,2.035000e-02


##### Sex 


In [6]:
g_sex <- mgcv::gam(y..has_mutation. ~ centered_log_phbrI * centered_log_phbrII * centered_sex, 
                   random=list(patient_ids=~1), 
                   family=binomial(link='logit'), 
                   data=discovery_data)
summary(g_sex)$p.table

,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),-6.59781340,0.02021266,-326.419910,0.000000e+00
centered_log_phbrI,0.04838672,0.01657149,2.919877,3.501693e-03
centered_log_phbrII,0.31947652,0.02016944,15.839632,1.657884e-56
centered_sex,-0.02197038,0.04143791,-0.530200,5.959733e-01
centered_log_phbrI:centered_log_phbrII,-0.02987515,0.01427653,-2.092606,3.638438e-02
centered_log_phbrI:centered_sex,0.07589780,0.03436274,2.208724,2.719386e-02
centered_log_phbrII:centered_sex,0.14913091,0.04174771,3.572194,3.540028e-04
centered_log_phbrI:centered_log_phbrII:centered_sex,-0.06652832,0.02996782,-2.219992,2.641929e-02


##### Odds ratio analysis
- Quantifies the influence of both PHBR-I and PHBR-II scores on probability of mutation using odds ratios between respective 25th and 75th percentiles

In [25]:
gam_compare = mgcv::gam(y..has_mutation. ~ s(centered_log_phbrI, centered_log_phbrII), 
                        data=discovery_data, 
                        random=list(patient_ids=~1), 
                        family='binomial')

In [26]:
low_x = quantile(discovery_data[['centered_log_phbrI']], 0.25, names=FALSE)
high_x = quantile(discovery_data[['centered_log_phbrI']], 0.75, names=FALSE)
low_z = quantile(discovery_data[['centered_log_phbrII']], 0.25, names=FALSE)
high_z = quantile(discovery_data[['centered_log_phbrII']], 0.75, names=FALSE)

In [27]:
results1 = or_gam(data=discovery_data, model=gam_compare, pred=c("centered_log_phbrI"), values=c(low_x, high_x))
results2 = or_gam(data=discovery_data, model=gam_compare, pred=c("centered_log_phbrII"), values=c(low_z, high_z))

In [28]:
OR <- CI_low <- CI_high <-  predicted <- vector("list",2)

In [29]:
OR[[1]] <- results1[['oddsratio']]
CI_low[[1]] <- results1[['CI_low (2.5%)']]
CI_high[[1]] <- results1[['CI_high (97.5%)']]
predicted[[1]] <- results1[['predictor']]

OR[[2]] <- results2[['oddsratio']]
CI_low[[2]] <- results2[['CI_low (2.5%)']]
CI_high[[2]] <- results2[['CI_high (97.5%)']]
predicted[[2]] <- results2[['predictor']]

In [30]:
cbind(cbind(cbind(OR, CI_low), CI_high), predicted)

OR,CI_low,CI_high,predicted
1.277088,1.265015,1.289277,centered_log_phbrI
3.404349,3.194265,3.62825,centered_log_phbrII


### Table S1

##### PHBR-I * sex * age

In [14]:
g_sex_age_I <- mgcv::gam(y..has_mutation. ~ centered_log_phbrI * centered_sex * centered_age, 
                          random=list(patient_ids=~1), 
                          family=binomial(link='logit'), 
                          data=discovery_data)
summary(g_sex_age_I)$p.table

,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),-6.5574332479,0.0181316512,-361.65670684,0.000000e+00
centered_log_phbrI,0.1327849415,0.0138231455,9.60598597,7.543248e-22
centered_sex,-0.0031770340,0.0370827598,-0.08567415,9.317255e-01
centered_age,-0.0026315978,0.0011740067,-2.24155266,2.499030e-02
centered_log_phbrI:centered_sex,0.0826691092,0.0282561037,2.92570802,3.436733e-03
centered_log_phbrI:centered_age,-0.0028545817,0.0008916852,-3.20133361,1.367931e-03
centered_sex:centered_age,0.0022298839,0.0023793197,0.93719391,3.486588e-01
centered_log_phbrI:centered_sex:centered_age,0.0008591691,0.0018053837,0.47589279,6.341508e-01


##### PHBR-II * sex * age

In [12]:
g_sex_age_II <- mgcv::gam(y..has_mutation. ~ centered_log_phbrII * centered_sex * centered_age, 
                          random=list(patient_ids=~1), 
                          family=binomial(link='logit'), 
                          data=discovery_data)
summary(g_sex_age_II)$p.table

,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),-6.612666670,0.019282675,-342.933051,0.000000e+00
centered_log_phbrII,0.332254644,0.018461598,17.997068,2.054090e-72
centered_sex,-0.049209919,0.039728871,-1.238644,2.154774e-01
centered_age,-0.001573477,0.001257245,-1.251528,2.107418e-01
centered_log_phbrII:centered_sex,0.162517556,0.038177967,4.256842,2.073352e-05
centered_log_phbrII:centered_age,-0.004354618,0.001205241,-3.613069,3.025937e-04
centered_sex:centered_age,0.003641782,0.002569918,1.417081,1.564593e-01
centered_log_phbrII:centered_sex:centered_age,-0.002502864,0.002469517,-1.013503,3.108198e-01


### 1D. Table S2: Without cancer types with significant mutational signature ratios

In [17]:
sig_mutsig_tumor_types = c('TCGA-LIHC', 'TCGA-GBM', 'TCGA-HNSC', 'TCGA-SKCM', 'TCGA-STAD')

# subset
discovery_data_mutsig = subset(discovery_data, !(disease %in% sig_mutsig_tumor_types))
cat(dim(discovery_data_mutsig))

1468698 16

##### Age

In [31]:
g_age_mutsig <- mgcv::gam(y..has_mutation. ~ centered_log_phbrI * centered_log_phbrII * centered_age, 
                          random=list(patient_ids=~1), 
                          family=binomial(link='logit'), 
                          data=discovery_data_mutsig)
summary(g_age_mutsig)#$p.table


Family: binomial 
Link function: logit 

Formula:
y..has_mutation. ~ centered_log_phbrI * centered_log_phbrII * 
    centered_age

Parametric coefficients:
                                                     Estimate Std. Error
(Intercept)                                         -6.596856   0.024783
centered_log_phbrI                                   0.058676   0.020325
centered_log_phbrII                                  0.330129   0.024815
centered_age                                        -0.002070   0.001525
centered_log_phbrI:centered_log_phbrII              -0.044378   0.017631
centered_log_phbrI:centered_age                     -0.004372   0.001265
centered_log_phbrII:centered_age                    -0.003960   0.001542
centered_log_phbrI:centered_log_phbrII:centered_age  0.002891   0.001093
                                                     z value Pr(>|z|)    
(Intercept)                                         -266.186  < 2e-16 ***
centered_log_phbrI                    

##### Sex

In [20]:
g_sex_mutsig <- mgcv::gam(y..has_mutation. ~ centered_log_phbrI * centered_log_phbrII * centered_sex, 
                          random=list(patient_ids=~1), 
                          family=binomial(link='logit'), 
                          data=discovery_data_mutsig)
summary(g_sex_mutsig)$p.table

,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),-6.59087120,0.02461869,-267.7182091,0.000000e+00
centered_log_phbrI,0.06297762,0.02040530,3.0863362,2.026396e-03
centered_log_phbrII,0.33397155,0.02476720,13.4844275,1.931589e-41
centered_sex,-0.03035719,0.05001406,-0.6069732,5.438687e-01
centered_log_phbrI:centered_log_phbrII,-0.04440234,0.01787309,-2.4843118,1.298021e-02
centered_log_phbrI:centered_sex,0.07544595,0.04189038,1.8010326,7.169774e-02
centered_log_phbrII:centered_sex,0.12534961,0.05073305,2.4707683,1.348232e-02
centered_log_phbrI:centered_log_phbrII:centered_sex,-0.06209653,0.03700765,-1.6779376,9.335928e-02


# 2. Validation cohort

In [22]:
validation_data = read.csv('../data/validation_data/gam_input.validation.age.2.csv')
head(validation_data)

y..has_mutation.,log_phbr_i,log_phbr_ii,phbr_i,phbr_ii,patient_ids,sex,age,centered_log_phbr_i,centered_log_phbr_ii,centered_sex,centered_age
0,0.4360090,1.2207901,1.5465227,3.389865,1,1,48,0.1070133,-1.4116194,0.5801282,-11.02132
0,1.0962891,1.0089661,2.9930387,2.742764,1,1,48,0.7672935,-1.6234434,0.5801282,-11.02132
0,0.1366724,1.9118222,1.1464525,6.765405,1,1,48,-0.1923233,-0.7205873,0.5801282,-11.02132
0,1.4222226,3.7839778,4.1463258,43.990679,1,1,48,1.0932269,1.1515683,0.5801282,-11.02132
0,-0.2307330,0.9104665,0.7939514,2.485482,1,1,48,-0.5597287,-1.7219430,0.5801282,-11.02132
0,0.9009311,4.1053750,2.4618944,60.665489,1,1,48,0.5719354,1.4729655,0.5801282,-11.02132


### Table S3

##### Age

In [35]:
validation_data = read.csv('../data/validation_data/gam_input.validation.age.2.csv')
g_age_val <- mgcv::gam(y..has_mutation. ~ centered_log_phbrI * centered_age+centered_log_phbrII * centered_age, 
                       random=list(patient_ids=~1), 
                       family=binomial(link='logit'), 
                       data=validation_data)
summary(g_age_val)


Family: binomial 
Link function: logit 

Formula:
y..has_mutation. ~ centered_log_phbrI * centered_age + centered_log_phbrII * 
    centered_age

Parametric coefficients:
                                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -3.482019   0.099720 -34.918   <2e-16 ***
centered_log_phbrI               -0.153878   0.089903  -1.712   0.0870 .  
centered_age                     -0.009375   0.006945  -1.350   0.1771    
centered_log_phbrII               0.287683   0.132762   2.167   0.0302 *  
centered_log_phbrI:centered_age  -0.007862   0.006651  -1.182   0.2371    
centered_age:centered_log_phbrII -0.009064   0.009360  -0.968   0.3329    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


R-sq.(adj) =  0.00143   Deviance explained =  1.2%
UBRE = -0.72145  Scale est. = 1         n = 3640

##### Sex

In [36]:
validation_data = read.csv('../data/validation_data/gam_input.validation.sex.2.csv')
g_sex_val <- mgcv::gam(y..has_mutation. ~ centered_log_phbrI * centered_sex+centered_log_phbrII * centered_sex, 
                       random=list(patient_ids=~1), 
                       family=binomial(link='logit'), 
                       data=validation_data)
summary(g_sex_val)


Family: binomial 
Link function: logit 

Formula:
y..has_mutation. ~ centered_log_phbrI * centered_sex + centered_log_phbrII * 
    centered_sex

Parametric coefficients:
                                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -3.50436    0.10020 -34.972   <2e-16 ***
centered_log_phbrI               -0.17303    0.09113  -1.899   0.0576 .  
centered_sex                      0.21348    0.20180   1.058   0.2901    
centered_log_phbrII               0.26725    0.13418   1.992   0.0464 *  
centered_log_phbrI:centered_sex  -0.09819    0.18305  -0.536   0.5917    
centered_sex:centered_log_phbrII  0.30335    0.27197   1.115   0.2647    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


R-sq.(adj) =  0.000736   Deviance explained = 0.943%
UBRE = -0.72825  Scale est. = 1         n = 3640